In [ ]:
import datetime

from autumn.tools.project import get_project, TimeSeriesSet
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

from autumn.tools.plots.utils import REF_DATE
from autumn.models.covid_19.constants import BASE_DATETIME
from autumn.tools.calibration.targets import get_target_series
import pandas as pd
from autumn.tools.utils.pretty import pretty_print
from summer.utils import ref_times_to_dti

## Specify the region

In [ ]:
region = "victoria"
project = get_project("covid_19", region)
targets_path = f"../../../autumn/projects/covid_19/victoria/victoria/targets.secret.json"
ts_set = TimeSeriesSet.from_file(targets_path)
model = project.run_baseline_model(project.param_set.baseline)
derived_df = model.get_derived_outputs_df()

In [ ]:
start_times = [sc_params.to_dict()["time"]["start"] for sc_params in project.param_set.scenarios]
scenarios = project.run_scenario_models(model, project.param_set.scenarios, start_times=start_times)
derived_dfs = [m.get_derived_outputs_df() for m in scenarios]

## Specify the outputs

In [ ]:
outputs = (
    "notifications", 
    "infection_deaths", 
    "hospital_admissions",
    "hospital_occupancy",
    "icu_admissions", 
    "icu_occupancy"
)
plot_left_date = datetime.date(2021, 8, 2)

## Plot the outputs

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 15), sharex="all")
fig.tight_layout(w_pad=1.5, h_pad=3.5)
for i_ax, axis in enumerate(axes.reshape(-1)):
    output_name = outputs[i_ax]
    output = ts_set.get(output_name)
    axis.plot(ref_times_to_dti(REF_DATE, model.times), derived_df[outputs[i_ax]])
    axis.plot(derived_dfs[0].index, derived_dfs[0][outputs[i_ax]])
    axis.scatter(ref_times_to_dti(REF_DATE, output.times), output.values, c="k")
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_title(output_name)

## Inspect the computed value processes around contact tracing

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharex="all")
fig.tight_layout(w_pad=1.5, h_pad=3.5)

derived_df["prop_all_traced"] = derived_df["prop_contacts_with_detected_index"] * derived_df["prop_detected_traced"]

computed_processes = ("prevalence", "prop_contacts_with_detected_index", "prop_detected_traced", "prop_all_traced")
for i_ax, axis in enumerate(axes.reshape(-1)):
    process = computed_processes[i_ax]
    axis.plot(derived_df.index, derived_df[process])
    axis.set_title(process)
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=plot_left_date)
    axis.set_ylim((0., 1.))